In [1]:
%load_ext autoreload
%autoreload 2
%aimport utils_1_1

import pandas as pd
import numpy as np
import altair as alt
from altair_saver import save
import datetime
import dateutil.parser
from os.path import join

from constants_1_1 import SITE_FILE_TYPES
from utils_1_1 import (
    get_site_file_paths,
    get_site_file_info,
    get_site_ids,
    get_visualization_subtitle,
    get_country_color_map,
)
from theme import apply_theme
from web import for_website

alt.data_transformers.disable_max_rows(); # Allow using rows more than 5000

In [ ]:
WAVE_COLOR = [
    '#D45E00', # '#BA4338', # early
    '#0072B2', # late
    'black'
]
STROKE = None
AXIS_SHOW = alt.Axis(grid=True, labels=True, ticks=True, domain=True, tickMinStep=1)
AXIS_HIDE_TITLE = alt.Axis(grid=True, labels=False, ticks=True, domain=True)
AXIS_HIDE = alt.Axis(grid=True, labels=False, ticks=False, domain=True)
data_release='2021-04-27'

# Define Function to Viaulize

In [3]:
"""
CATEGORIES WE USE
"""
AGE_GROUPS = ['0-25', '26-49', '50-69', '70-79', '80+']
SEX_GROUPS = ['Female', 'Male']
RACE_GROUPS = ['White', 'Black']

def FUNC_DEMOGRAPHICS_BY_WAVE(
    _data, 
    country='', 
    race=True, 
    patient_group='all', # either 'all' or 'severe',
    ranges=None # e.g., [1, 1000, 0.9]
):
    d = _data.copy()
    d = d[d.severity == patient_group]
    
    """
    RENAME COLUMNS AND VALUES
    """
#     d = d.rename(columns={
#         'p.all': 'p_all', 
#         'n.all': 'n_all',
#         'p.severe': 'p_severe',
#         'n.severe': 'n_severe',
        
#         'p.all.lwr.ci': 'p_all_ci_l',
#         'p.all.upr.ci': 'p_all_ci_u',
#         'p.ever.lwr.ci': 'p_severe_ci_l',
#         'p.ever.upr.ci': 'p_severe_ci_u',
#         'p.all.sig': 'p_all_sig',
#         'p.ever.sig': 'p_severe_sig',
        
#         'p.lwr.ci': 'p_ci_l',
#         'p.upr.ci': 'p_ci_u',
#         'p.sig': 'p_sig'
#     })
# #     print(d)
#     d.group = d.group.apply(
#         lambda x: {
#             '00to25': '0-25',
#             '26to49': '26-49',
#             '50to69': '50-69',
#             '70to79': '70-79',
#             '80plus': '80+',
#             'female': 'Female',
#             'male': 'Male',
#             'white': 'White',
#             'black': 'Black',
#             'other': 'Other',
#             'other_age': 'Other',
#             'other_sex': 'Other',
#             'other_race': 'Other'
#         }[x]
#     )
#     d.wave = d.wave.apply(
#         lambda x: {
#             'early': 'Early',
#             'late': 'Late'
#         }[x]
#     )
    
    """
    /////////////////////
    SUB-CHARTS FOR GROUPS
    /////////////////////
    """
    
    """
    COMMON VISUAL PARAMETERS
    """
    width = 180
    titleX = -60
    top_panel_y_title = "Prevalence Among All Patients"
    mid_panel_y_title = "Severity Risk"
    
    """
    AGE GROUPS
    """
    ad = d[d.group.isin(AGE_GROUPS)]
    
    ############## Bar Chart for % of Participants ##############
    age_p_bar_base = alt.Chart(
        ad
    ).encode(
        x=alt.X(
            'group:N', 
            title=None,
            axis=AXIS_HIDE_TITLE
        ),
        y=alt.Y(f'pp:Q', axis=alt.Axis(format='.0%', titleX=titleX), title=top_panel_y_title, scale=alt.Scale(domain=[0, ranges[0]]) if ranges != None else alt.Scale()),
#         color=alt.Color("wave:N", scale=alt.Scale(domain=['Early', 'Late', 'Late - Early'], range=WAVE_COLOR), title="Wave")
        color=alt.Color("wave:N", scale=alt.Scale(domain=['First', 'Second'], range=WAVE_COLOR), title="Wave")
    ).properties(
        width=360, height=200
    )

    age_p_bar_early = age_p_bar_base.transform_filter(
        {'field': 'wave', 'oneOf': ['First']}
    ).mark_bar(
        xOffset=-10,
        size=20, stroke=STROKE
    )
    age_p_bar_late = age_p_bar_base.transform_filter(
        {'field': 'wave', 'oneOf': ['Second']}
    ).mark_bar(
        xOffset=10,
        size=20, stroke=STROKE
    )
    
    
    """
    ================================================================================================================================================
    ################################################################## SIG NOTATATION ##############################################################
    ================================================================================================================================================
    """
    age_p_error_bar_early = age_p_bar_early.mark_bar(
        size=2, color='black', opacity=1, xOffset=-10, 
    ).encode(
        x=alt.X('group:N', title=None, axis=AXIS_HIDE_TITLE),
        y=alt.Y(f'ci_u:Q'),
        y2=alt.Y2(f'ci_l:Q'),
        color=alt.value('black')
    )
    
    age_p_error_bar_late = age_p_bar_late.mark_bar(
        size=2, color='black', opacity=1, xOffset=10, 
    ).encode(
        x=alt.X('group:N', title=None, axis=AXIS_HIDE_TITLE),
        y=alt.Y(f'ci_u:Q'),
        y2=alt.Y2(f'ci_l:Q'),
        color=alt.value('black')
    )
    
    age_p_sig_tick = age_p_bar_base.mark_tick(
        size=45, yOffset=-10, strokeWidth=8
    ).encode(
        x=alt.X(
            'group:N',
            title=None,
            axis=AXIS_HIDE_TITLE
        ),
        y=alt.Y(f'max(pp):Q', axis=alt.Axis(format='.0%', titleX=titleX), title=top_panel_y_title),
        color=alt.value('black')
    ).transform_filter(
        {'field': f"sig", 'oneOf': [True]}
    )
    
    age_p_sig_star = age_p_sig_tick.mark_text(
        yOffset=-15, fontSize=20
    ).encode(
        x=alt.X(
            'group:N',
            title=None,
            axis=AXIS_HIDE_TITLE
        ),
        y=alt.Y(f'max(pp):Q', axis=alt.Axis(format='.0%', titleX=titleX), title=top_panel_y_title),
        color=alt.value('black'),
        text=alt.value('*')
    )
    
    age_p_bar = alt.layer(age_p_bar_early, age_p_bar_late, age_p_error_bar_early, age_p_error_bar_late, age_p_sig_tick, age_p_sig_star)

    ############## Bar Chart for # of Participants ##############
    age_n_bar_base = alt.Chart(
        ad
    ).encode(
        x=alt.X(
            'group:N', 
            title=None, 
            axis=AXIS_HIDE
        ),
        y=alt.Y(f'x:Q', title="# Patients", axis=alt.Axis(titleX=titleX), scale=alt.Scale(domain=[0, ranges[1]]) if ranges != None else alt.Scale()), # scale=alt.Scale(domain=[0, 40000])),
        color=alt.Color("wave:N", scale=alt.Scale(domain=['First', 'Second'], range=WAVE_COLOR), title="Wave")
    ).properties(
        width=360, height=200
    )

    age_n_bar_early = age_n_bar_base.transform_filter(
        {'field': 'wave', 'oneOf': ['First']}
    ).mark_bar(
        xOffset=-10,
        size=20, stroke=STROKE
    )
    age_n_bar_late = age_n_bar_base.transform_filter(
        {'field': 'wave', 'oneOf': ['Second']}
    ).mark_bar(
        xOffset=10,
        size=20, stroke=STROKE
    )
    
    age_n_bar = alt.layer(age_n_bar_early, age_n_bar_late)
    
    ############## Bar Chart for % of Ever Severe ##############
    age_s_bar_base = alt.Chart(
        ad
    ).encode(
        x=alt.X(
            'group:N', 
            title='Age', 
            axis=AXIS_SHOW
        ),
        y=alt.Y(f'pp_severity_risk:Q', title=mid_panel_y_title, axis=alt.Axis(format=".0%", titleX=titleX), scale=alt.Scale(domain=[0, ranges[0]]) if ranges != None else alt.Scale()),
        color=alt.Color("wave:N", scale=alt.Scale(domain=['First', 'Second'], range=WAVE_COLOR), title="Wave")
    ).properties(
        width=360, height=200
    )

    age_s_bar_early = age_s_bar_base.transform_filter(
        {'field': 'wave', 'oneOf': ['First']}
    ).mark_bar(
        xOffset=-10,
        size=20, stroke=STROKE
    )
    age_s_bar_late = age_s_bar_base.transform_filter(
        {'field': 'wave', 'oneOf': ['Second']}
    ).mark_bar(
        xOffset=10,
        size=20, stroke=STROKE
    )
    
    """
    ================================================================================================================================================
    ################################################################## SIG NOTATATION ##############################################################
    ================================================================================================================================================
    """
    age_s_error_bar_early = age_s_bar_early.mark_bar(
        size=2, color='black', opacity=1, xOffset=-10, 
    ).encode(
        x=alt.X('group:N'),
        y=alt.Y(f'ci_u_severity_risk:Q'),
        y2=alt.Y2(f'ci_l_severity_risk:Q'),
        color=alt.value('black')
    )
    
    age_s_error_bar_late = age_s_bar_late.mark_bar(
        size=2, color='black', opacity=1, xOffset=10, 
    ).encode(
        x=alt.X('group:N'),
        y=alt.Y(f'ci_u_severity_risk:Q'),
        y2=alt.Y2(f'ci_l_severity_risk:Q'),
        color=alt.value('black')
    )
    
    age_s_sig_tick = age_s_bar_base.mark_tick(
        size=45, yOffset=-10, strokeWidth=8
    ).encode(
        x=alt.X(
            'group:N',

        ),
        y=alt.Y(f'max(ci_u_severity_risk):Q', axis=alt.Axis(format='.0%')),
        color=alt.value('black')
    ).transform_filter(
        {'field': f"sig_severity_risk", 'oneOf': [True]}
    )
    
    age_s_sig_star = age_s_sig_tick.mark_text(
        yOffset=-15, fontSize=20
    ).encode(
        x=alt.X(
            'group:N',

        ),
        y=alt.Y(f'max(ci_u_severity_risk):Q'),
        color=alt.value('black'),
        text=alt.value('*')
    )
    
    age_s_bar = alt.layer(age_s_bar_early, age_s_bar_late, age_s_error_bar_early, age_s_error_bar_late, age_s_sig_tick, age_s_sig_star)
    
    """
    SEX GROUPS
    """    
    sd = d[d.group.isin(SEX_GROUPS)]

    ############## Bar Chart for % of Participants ##############
    sex_p_bar_base = alt.Chart(
        sd
    ).encode(
        x=alt.X(
            'group:N', 
            title=None, 
            axis=AXIS_HIDE_TITLE
        ),
        y=alt.Y(f'pp:Q', axis=alt.Axis(format='.0%', grid=True, labels=False, ticks=False, domain=False), title=None),
        color=alt.Color("wave:N", scale=alt.Scale(domain=['First', 'Second'], range=WAVE_COLOR), title="Wave")
    ).properties(
        width=width, height=200
    )

    sex_p_bar_early = sex_p_bar_base.transform_filter(
        {'field': 'wave', 'oneOf': ['First']}
    ).mark_bar(
        xOffset=-10,
        size=20, stroke=STROKE
    )
    sex_p_bar_late = sex_p_bar_base.transform_filter(
        {'field': 'wave', 'oneOf': ['Second']}
    ).mark_bar(
        xOffset=10,
        size=20, stroke=STROKE
    )

    """
    !!! ERROR BARS !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    """
    sex_p_error_bar_early = sex_p_bar_early.mark_bar(
        size=2, color='black', opacity=1, xOffset=-10, 
    ).encode(
        x=alt.X('group:N', title=None, axis=AXIS_HIDE_TITLE),
        y=alt.Y(f'ci_u:Q'),
        y2=alt.Y2(f'ci_l:Q'),
        color=alt.value('black')
    )
    
    sex_p_error_bar_late = sex_p_bar_late.mark_bar(
        size=2, color='black', opacity=1, xOffset=10, 
    ).encode(
        x=alt.X('group:N', title=None, axis=AXIS_HIDE_TITLE),
        y=alt.Y(f'ci_u:Q'),
        y2=alt.Y2(f'ci_l:Q'),
        color=alt.value('black')
    )
    
    """
    ================================================================================================================================================
    ################################################################## SIG NOTATATION ##############################################################
    ================================================================================================================================================
    """
    sex_p_sig_tick = sex_p_bar_base.mark_tick(
        size=45, yOffset=-10, strokeWidth=8
    ).encode(
        x=alt.X(
            'group:N',
            title=None,
            axis=AXIS_HIDE_TITLE
        ),
        y=alt.Y(f'max(pp):Q', axis=alt.Axis(format='.0%', titleX=titleX), title=top_panel_y_title),
        color=alt.value('black')
    ).transform_filter(
        {'field': f"sig", 'oneOf': [True]}
    )
    
    sex_p_sig_star = sex_p_sig_tick.mark_text(
        yOffset=-15, fontSize=20
    ).encode(
        x=alt.X(
            'group:N',
            title=None,
            axis=AXIS_HIDE_TITLE
        ),
        y=alt.Y(f'max(pp):Q', axis=alt.Axis(format='.0%', titleX=titleX), title=top_panel_y_title),
        color=alt.value('black'),
        text=alt.value('*')
    )
    
    sex_p_bar = alt.layer(sex_p_bar_early, sex_p_bar_late, sex_p_error_bar_early, sex_p_error_bar_late, sex_p_sig_tick, sex_p_sig_star)

    ############## Bar Chart for # of Participants ##############
    sex_n_bar_base = alt.Chart(
        sd
    ).encode(
        x=alt.X(
            'group:N', 
            title=None, 
            axis=AXIS_HIDE
        ),
        y=alt.Y(f'x:Q', axis=alt.Axis(title=None, grid=True, labels=False, ticks=False, domain=False), title=None),
        color=alt.Color("wave:N", scale=alt.Scale(domain=['First', 'Second'], range=WAVE_COLOR), title="Wave")
    ).properties(
        width=width, height=200
    )

    sex_n_bar_early = sex_n_bar_base.transform_filter(
        {'field': 'wave', 'oneOf': ['First']}
    ).mark_bar(
        xOffset=-10,
        size=20, stroke=STROKE
    )
    sex_n_bar_late = sex_n_bar_base.transform_filter(
        {'field': 'wave', 'oneOf': ['Second']}
    ).mark_bar(
        xOffset=10,
        size=20, stroke=STROKE
    )
    sex_n_bar = alt.layer(sex_n_bar_early, sex_n_bar_late)
    
    ############## Bar Chart for % of Ever Severe ##############
    sex_s_bar_base = alt.Chart(
        sd
    ).encode(
        x=alt.X(
            'group:N', 
            title='Sex', 
            axis=AXIS_SHOW
        ),
        y=alt.Y(f'pp_severity_risk:Q', axis=alt.Axis(title=None, grid=True, labels=False, ticks=False, domain=False), title=None),
        color=alt.Color("wave:N", scale=alt.Scale(domain=['First', 'Second'], range=WAVE_COLOR), title="Wave")
    ).properties(
        width=width, height=200
    )

    sex_s_bar_early = sex_s_bar_base.transform_filter(
        {'field': 'wave', 'oneOf': ['First']}
    ).mark_bar(
        xOffset=-10,
        size=20, stroke=STROKE
    )
    sex_s_bar_late = sex_s_bar_base.transform_filter(
        {'field': 'wave', 'oneOf': ['Second']}
    ).mark_bar(
        xOffset=10,
        size=20, stroke=STROKE
    )

    """
    ================================================================================================================================================
    ################################################################## SIG NOTATATION ##############################################################
    ================================================================================================================================================
    """
    sex_s_error_bar_early = sex_s_bar_early.mark_bar(
        size=2, color='black', opacity=1, xOffset=-10, 
    ).encode(
        x=alt.X('group:N'),
        y=alt.Y(f'ci_u_severity_risk:Q'),
        y2=alt.Y2(f'ci_l_severity_risk:Q'),
        color=alt.value('black')
    )
    
    sex_s_error_bar_late = sex_s_bar_late.mark_bar(
        size=2, color='black', opacity=1, xOffset=10, 
    ).encode(
        x=alt.X('group:N'),
        y=alt.Y(f'ci_u_severity_risk:Q'),
        y2=alt.Y2(f'ci_l_severity_risk:Q'),
        color=alt.value('black')
    )
    
    sex_s_sig_tick = sex_s_bar_base.mark_tick(
        size=45, yOffset=-10, strokeWidth=8
    ).encode(
        x=alt.X(
            'group:N'
        ),
        y=alt.Y(f'max(ci_u_severity_risk):Q', axis=alt.Axis(format='.0%')),
        color=alt.value('black')
    ).transform_filter(
        {'field': f"sig_severity_risk", 'oneOf': [True]}
    )
    
    sex_s_sig_star = sex_s_sig_tick.mark_text(
        yOffset=-15, fontSize=20
    ).encode(
        x=alt.X(
            'group:N'
        ),
        y=alt.Y(f'max(ci_u_severity_risk):Q'),
        color=alt.value('black'),
        text=alt.value('*')
    )
    
    sex_s_bar = alt.layer(sex_s_bar_early, sex_s_bar_late, sex_s_error_bar_early, sex_s_error_bar_late, sex_s_sig_tick, sex_s_sig_star)
    
    """
    RACE GROUPS
    """    
    rd = d[d.group.isin(RACE_GROUPS)]

    ############## Bar Chart for % of Participants ##############
    race_p_bar_base = alt.Chart(
        rd
    ).encode(
        x=alt.X(
            'group:N', 
            title=None, 
            axis=AXIS_HIDE_TITLE
        ),
        y=alt.Y(f'pp:Q', axis=alt.Axis(format='.0%', grid=True, labels=False, ticks=False, domain=False), title=None),
        color=alt.Color("wave:N", scale=alt.Scale(domain=['First', 'Second'], range=WAVE_COLOR), title="Wave", legend=alt.Legend(title="Wave"))
    ).properties(
        width=width, height=200
    )

    race_p_bar_early = race_p_bar_base.transform_filter(
        {'field': 'wave', 'oneOf': ['First']}
    ).mark_bar(
        xOffset=-10,
        size=20, stroke=STROKE
    )
    race_p_bar_late = race_p_bar_base.transform_filter(
        {'field': 'wave', 'oneOf': ['Second']}
    ).mark_bar(
        xOffset=10,
        size=20, stroke=STROKE
    )
    
    """
    !!! ERROR BARS !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    """
    race_p_error_bar_early = race_p_bar_early.mark_bar(
        size=2, color='black', opacity=1, xOffset=-10, 
    ).encode(
        x=alt.X('group:N', title=None, axis=AXIS_HIDE_TITLE),
        y=alt.Y(f'ci_u:Q', title=None),
        y2=alt.Y2(f'ci_l:Q'),
        color=alt.value('black')
    )
    
    race_p_error_bar_late = race_p_bar_late.mark_bar(
        size=2, color='black', opacity=1, xOffset=10, 
    ).encode(
        x=alt.X('group:N', title=None, axis=AXIS_HIDE_TITLE),
        y=alt.Y(f'ci_u:Q', title=None),
        y2=alt.Y2(f'ci_l:Q'),
        color=alt.value('black')
    )

    
    """
    ================================================================================================================================================
    ################################################################## SIG NOTATATION ##############################################################
    ================================================================================================================================================
    """
    race_p_sig_tick = race_p_bar_base.mark_tick(
        size=45, yOffset=-10, strokeWidth=8
    ).encode(
        x=alt.X(
            'group:N',
            title=None,
            axis=AXIS_HIDE_TITLE
        ),
        y=alt.Y(f'max(pp):Q', axis=alt.Axis(format='.0%', titleX=titleX), title="Percentage of Patients"),
        color=alt.value('black')
    ).transform_filter(
        {'field': f"sig", 'oneOf': [True]}
    )
    
    race_p_sig_star = race_p_sig_tick.mark_text(
        yOffset=-15, fontSize=20
    ).encode(
        x=alt.X(
            'group:N',
            title=None,
            axis=AXIS_HIDE_TITLE
        ),
        y=alt.Y(f'max(pp):Q', axis=alt.Axis(format='.0%', titleX=titleX), title="Percentage of Patients"),
        color=alt.value('black'),
        text=alt.value('*')
    )
    
    race_p_bar = alt.layer(race_p_bar_early, race_p_bar_late, race_p_error_bar_early, race_p_sig_tick, race_p_sig_star, race_p_error_bar_late)

    ############## Bar Chart for # of Participants ##############
    race_n_bar_base = alt.Chart(
        rd
    ).encode(
        x=alt.X(
            'group:N', 
            title=None, 
            axis=AXIS_HIDE
        ),
        y=alt.Y(f'x:Q', axis=alt.Axis(title=None, grid=True, labels=False, ticks=False, domain=False), title=None),
        color=alt.Color("wave:N", scale=alt.Scale(domain=['First', 'Second'], range=WAVE_COLOR), title="Wave")
    ).properties(
        width=width, height=200
    )

    race_n_bar_early = race_n_bar_base.transform_filter(
        {'field': 'wave', 'oneOf': ['First']}
    ).mark_bar(
        xOffset=-10,
        size=20, stroke=STROKE
    )
    race_n_bar_late = race_n_bar_base.transform_filter(
        {'field': 'wave', 'oneOf': ['Second']}
    ).mark_bar(
        xOffset=10,
        size=20, stroke=STROKE
    )
    race_n_bar = alt.layer(race_n_bar_early, race_n_bar_late)

    ############## Bar Chart for % of Ever Severe ##############
    race_s_bar_base = alt.Chart(
        rd
    ).encode(
        x=alt.X(
            'group:N', 
            title='Race', 
            axis=AXIS_SHOW
        ),
        y=alt.Y(f'pp_severity_risk:Q', axis=alt.Axis(title=None, grid=True, labels=False, ticks=False, domain=False), title=None),
        color=alt.Color("wave:N", scale=alt.Scale(domain=['First', 'Second'], range=WAVE_COLOR), title="Wave")
    ).properties(
        width=width, height=200
    )

    race_s_bar_early = race_s_bar_base.transform_filter(
        {'field': 'wave', 'oneOf': ['First']}
    ).mark_bar(
        xOffset=-10,
        size=20, stroke=STROKE
    )
    race_s_bar_late = race_s_bar_base.transform_filter(
        {'field': 'wave', 'oneOf': ['Second']}
    ).mark_bar(
        xOffset=10,
        size=20, stroke=STROKE
    )
    
    """
    ================================================================================================================================================
    ################################################################## SIG NOTATATION ##############################################################
    ================================================================================================================================================
    """
    race_s_error_bar_early = race_s_bar_early.mark_bar(
        size=2, color='black', opacity=1, xOffset=-10, 
    ).encode(
        x=alt.X('group:N'),
        y=alt.Y(f'ci_u_severity_risk:Q'),
        y2=alt.Y2(f'ci_l_severity_risk:Q'),
        color=alt.value('black')
    )
    
    race_s_error_bar_late = race_s_bar_late.mark_bar(
        size=2, color='black', opacity=1, xOffset=10, 
    ).encode(
        x=alt.X('group:N'),
        y=alt.Y(f'ci_u_severity_risk:Q'),
        y2=alt.Y2(f'ci_l_severity_risk:Q'),
        color=alt.value('black')
    )
    
    race_s_sig_tick = race_s_bar_base.mark_tick(
        size=45, yOffset=-10, strokeWidth=8
    ).encode(
        x=alt.X(
            'group:N'
        ),
        y=alt.Y(f'max(ci_u_severity_risk):Q', axis=alt.Axis(format='.0%')),
        color=alt.value('black')
    ).transform_filter(
        {'field': f"sig_severity_risk", 'oneOf': [True]}
    )
    
    race_s_sig_star = race_s_sig_tick.mark_text(
        yOffset=-15, fontSize=20
    ).encode(
        x=alt.X(
            'group:N'
        ),
        y=alt.Y(f'max(ci_u_severity_risk):Q'),
        color=alt.value('black'),
        text=alt.value('*')
    )
    
    race_s_bar = alt.layer(race_s_bar_early, race_s_bar_late, race_s_error_bar_early, race_s_error_bar_late, race_s_sig_tick, race_s_sig_star)
    
    """
    /////////////////////
    LAYER-CHARTS FOR DIFF
    /////////////////////
    """
#     diff = d.copy()
#     diff.loc[diff.wave == 'Early', 'p_all'] = diff[diff.wave == 'Early'].p_all.apply(lambda x: -x)
#     diff = diff.groupby(['group']).sum()
#     diff = diff.reset_index()

#     # VISUAL PARAMETERS
#     xOffset = -7

#     """
#     AGE DIFF
#     """
#     age_diff = diff[diff.group.isin(AGE_GROUPS)]

#     age_diff_chart = alt.Chart(
#         age_diff
#     ).mark_tick(
#         color="black", size=40, stroke="white", strokeWidth=2, thickness=5, xOffset=xOffset
#     ).encode(
#         x=alt.X("group:N", title=None, axis=None),
#         y=alt.Y("p_all:Q")
#     )

#     """
#     SEX DIFF
#     """
#     sex_diff = diff[diff.group.isin(SEX_GROUPS)]

#     sex_diff_chart = alt.Chart(
#         sex_diff
#     ).mark_tick(
#         color="black", size=40, stroke="white", strokeWidth=2, thickness=5, xOffset=xOffset
#     ).encode(
#         x=alt.X("group:N", title=None, axis=None),
#         y=alt.Y("p_all:Q")
#     )

#     """
#     RACE DIFF
#     """
#     race_diff = diff[diff.group.isin(RACE_GROUPS)]

#     race_diff_chart = alt.Chart(
#         race_diff
#     ).mark_tick(
#         color="black", size=40, stroke="white", strokeWidth=2, thickness=5, xOffset=xOffset
#     ).encode(
#         x=alt.X("group:N", title=None, axis=None),
#         y=alt.Y("p_all:Q")
#     )
    
    """
    ////////////////////////////////////
    ASSEMBLE
    ////////////////////////////////////
    """
    if race:
        final_chart = alt.hconcat(
            (age_p_bar),# + age_diff_chart), 
            (sex_p_bar),# + sex_diff_chart), 
            (race_p_bar),# + race_diff_chart)
            spacing=10,
        ).resolve_scale(y='shared', color='shared')
    
        nfinal_chart = alt.hconcat(age_n_bar, sex_n_bar, race_n_bar, spacing=10).resolve_scale(y='shared', color='shared')
        sfinal_chart = alt.hconcat(age_s_bar, sex_s_bar, race_s_bar, spacing=10).resolve_scale(y='shared', color='shared')
    else:
        # Because we do not show race info for the countries other than USA
        final_chart = alt.hconcat(
            (age_p_bar),# + age_diff_chart), 
            (sex_p_bar),# + sex_diff_chart)
            spacing=10
        ).resolve_scale(y='shared', color='shared')
    
        nfinal_chart = alt.hconcat(age_n_bar, sex_n_bar, spacing=10).resolve_scale(y='shared', color='shared')
        sfinal_chart = alt.hconcat(age_s_bar, sex_s_bar, spacing=10).resolve_scale(y='shared', color='shared')

    final_chart = (
        # (final_chart & sfinal_chart & nfinal_chart).properties( # 
        alt.vconcat(final_chart, sfinal_chart, spacing=10).properties(
        # alt.vconcat(final_chart, nfinal_chart, sfinal_chart, spacing=10).properties(
            title={
                "text": f"{country} Demographics Of {'All' if patient_group == 'all' else 'Ever Severe'} Patients By Wave",
                "dx": 80,
#                 "subtitle": get_visualization_subtitle(data_release=data_release, with_num_sites=False), 
                "subtitleColor": "gray",
            }
        )
    )

#     final_chart = apply_theme(
#         final_chart,
#         axis_y_title_font_size=16,
#         title_anchor='start',
#         legend_orient='right'
#     )
    return final_chart

# Demographics

In [4]:
df2 = pd.read_csv(join("..", "data", "1.1.resurgence", "demographics", "demographics_stats_NO_VA.csv"))
df2_va = pd.read_csv(join("..", "data", "1.1.resurgence", "demographics", "demographics_stats.csv"))
cdf2 = pd.read_csv(join("..", "data", "1.1.resurgence", "demographics", "demographics_stats_bycountry_NO_VA.csv"))

df2 = df2.rename(columns={
    'demo_group': 'group',
    'ci.95L': 'ci_l',
    'ci.95U': 'ci_u',
})
df2_va = df2_va.rename(columns={
    'demo_group': 'group',
    'ci.95L': 'ci_l',
    'ci.95U': 'ci_u',
})
cdf2 = cdf2.rename(columns={
    'demo_group': 'group',
    'ci.95L': 'ci_l',
    'ci.95U': 'ci_u',
})


df2['severity'] = df2.group.apply(lambda x: 'all' if '_all_' in x else 'severe')
df2_va['severity'] = df2_va.group.apply(lambda x: 'all' if '_all_' in x else 'severe')
cdf2['severity'] = cdf2.group.apply(lambda x: 'all' if '_all_' in x else 'severe')

df2.group = df2.group.apply(lambda x: x.replace('num_patients_all_', '').replace('num_patients_ever_severe_', '').replace('.risk', '').replace('.rr', ''))
df2_va.group = df2_va.group.apply(lambda x: x.replace('num_patients_all_', '').replace('num_patients_ever_severe_', '').replace('.risk', '').replace('.rr', ''))
cdf2.group = cdf2.group.apply(lambda x: x.replace('num_patients_all_', '').replace('num_patients_ever_severe_', '').replace('.risk', '').replace('.rr', ''))

df2.group = df2.group.apply(
    lambda x: {
        '00to25': '0-25',
        '26to49': '26-49',
        '50to69': '50-69',
        '70to79': '70-79',
        '80plus': '80+',
        'female': 'Female',
        'male': 'Male',
        'white': 'White',
        'black': 'Black',
        'other': 'Other',
        'other_age': 'Other',
        'other_sex': 'Other',
        'other_race': 'Other'
    }[x]
)
df2.wave = df2.wave.apply(
    lambda x: {
        'early': 'First',
        'late': 'Second'
    }[x] if x == x else None
)
df2_va.group = df2_va.group.apply(
    lambda x: {
        '00to25': '0-25',
        '26to49': '26-49',
        '50to69': '50-69',
        '70to79': '70-79',
        '80plus': '80+',
        'female': 'Female',
        'male': 'Male',
        'white': 'White',
        'black': 'Black',
        'other': 'Other',
        'other_age': 'Other',
        'other_sex': 'Other',
        'other_race': 'Other'
    }[x]
)
df2_va.wave = df2_va.wave.apply(
    lambda x: {
        'early': 'First',
        'late': 'Second'
    }[x] if x == x else None
)
cdf2.group = cdf2.group.apply(
    lambda x: {
        '00to25': '0-25',
        '26to49': '26-49',
        '50to69': '50-69',
        '70to79': '70-79',
        '80plus': '80+',
        'female': 'Female',
        'male': 'Male',
        'white': 'White',
        'black': 'Black',
        'other': 'Other',
        'other_age': 'Other',
        'other_sex': 'Other',
        'other_race': 'Other'
    }[x]
)
cdf2.wave = cdf2.wave.apply(
    lambda x: {
        'early': 'First',
        'late': 'Second'
    }[x] if x == x else None
)

df2 = df2[
    (df2.group != 'White') &
    (df2.group != 'Black')
]
df2_va = df2_va[
    (df2_va.group != 'White') &
    (df2_va.group != 'Black')
]
cdf2 = cdf2[
    (cdf2.group == 'White') | (cdf2.group == 'Black')
]
df2 = df2.append(cdf2)
df2_va = df2_va.append(cdf2)

# FILTER
# df2 = df2[df2['severity'] == 'all']

# df2 = df2[df2.effect_size != 'risk_ratio'].copy()
# pdf = df2[df2.effect_size == 'prevalence'].copy()

# pdf = df2[(df2.effect_size == 'prevalence') & (df2.severity == 'all')].copy()
# sdf = df2[df2.effect_size == 'severity_risk'].copy().drop(columns=['x', 'n' , 'severity', 'effect_size']).rename(columns={'pp': 'pp_severity_risk', 'sig': 'sig_severity_risk', 'ci_l': 'ci_l_severity_risk', 'ci_u': 'ci_u_severity_risk'})

# pdf = pdf.set_index(['wave', 'group', 'country'])
# sdf = sdf.set_index(['wave', 'group', 'country'])

# df2 = pd.concat([pdf, sdf], axis=1).reset_index()

# print(df2)

# pdf = df2[(df2.effect_size == 'prevalence') & (df2.severity == 'all')].copy()
# sdf = df2[df2.effect_size == 'severity_risk'].copy().drop(columns=['x', 'n' , 'severity', 'effect_size']).rename(columns={'pp': 'pp_severity_risk', 'sig': 'sig_severity_risk', 'ci_l': 'ci_l_severity_risk', 'ci_u': 'ci_u_severity_risk'})

# pdf = pdf.set_index(['wave', 'group', 'country'])
# sdf = sdf.set_index(['wave', 'group', 'country'])

# df2 = pd.concat([pdf, sdf], axis=1).reset_index().copy()

df2_va

,country,wave,effect_size,group,pp,ci_l,ci_u,x,n,sig,severity
0,ALL,First,prevalence,0-25,0.011661,0.004962,0.027158,791.0,32452.0,False,all
1,ALL,Second,prevalence,0-25,0.011374,0.003864,0.033000,1284.0,47035.0,False,all
2,ALL,First,prevalence,26-49,0.153016,0.125024,0.185942,5418.0,32452.0,False,all
3,ALL,Second,prevalence,26-49,0.151275,0.121390,0.186952,6662.0,47035.0,False,all
4,ALL,First,prevalence,50-69,0.376258,0.357617,0.395273,12046.0,32452.0,True,all
...,...,...,...,...,...,...,...,...,...,...,...
289,USA,Second,severity_risk,Black,0.458631,0.369932,0.550032,1220.0,3056.0,False,severe
292,USA,First,severity_risk,White,0.485527,0.393781,0.578258,1989.5,4163.0,False,severe
293,USA,Second,severity_risk,White,0.436468,0.341190,0.536679,3976.0,10662.0,False,severe
301,USA,None,risk_ratio,Black,0.860053,0.734619,1.006905,NaN,NaN,False,severe


In [5]:
# for i, patient_group in enumerate(['all', 'severe']):
for i, patient_group in enumerate(['all']):
    pdf = df2[(df2.effect_size == 'prevalence') & (df2.severity == patient_group)].copy()
    sdf = df2_va[df2_va.effect_size != 'risk_ratio']
    sdf = sdf[sdf.effect_size == 'severity_risk'].copy().drop(columns=['x', 'n' , 'severity', 'effect_size']).rename(columns={'pp': 'pp_severity_risk', 'sig': 'sig_severity_risk', 'ci_l': 'ci_l_severity_risk', 'ci_u': 'ci_u_severity_risk'})

    pdf = pdf.set_index(['wave', 'group', 'country'])
    sdf = sdf.set_index(['wave', 'group', 'country'])

    df2_ = pd.concat([pdf, sdf], axis=1).reset_index().copy()

    if i == 0:
        plot = FUNC_DEMOGRAPHICS_BY_WAVE(df2_, country="", patient_group=patient_group)
    else:
        plot = alt.hconcat(plot, FUNC_DEMOGRAPHICS_BY_WAVE(df2_, country="", patient_group=patient_group), spacing=30)#.resolve_scale(color='independent')

plot = apply_theme(
    plot,
    axis_y_title_font_size=16,
    title_anchor='start',
    legend_orient='right'
)
plot

# df2
# pdf
# sdf
# df2_[df2_.sig == True]

alt.VConcatChart(...)

# Demographics by Country

In [7]:
df = pd.read_csv(join("..", "data", "1.1.resurgence", "demographics", "demographics_stats_bycountry_NO_VA.csv"))


df = df.rename(columns={
    'demo_group': 'group',
    'ci.95L': 'ci_l',
    'ci.95U': 'ci_u',
})

df['severity'] = df.group.apply(lambda x: 'all' if '_all_' in x else 'severe')

df.group = df.group.apply(lambda x: x.replace('num_patients_all_', '').replace('num_patients_ever_severe_', '').replace('.risk', '').replace('.rr', ''))

df.group = df.group.apply(
    lambda x: {
        '00to25': '0-25',
        '26to49': '26-49',
        '50to69': '50-69',
        '70to79': '70-79',
        '80plus': '80+',
        'female': 'Female',
        'male': 'Male',
        'white': 'White',
        'black': 'Black',
        'other': 'Other',
        'other_age': 'Other',
        'other_sex': 'Other',
        'other_race': 'Other'
    }[x]
)
df.wave = df.wave.apply(
    lambda x: {
        'early': 'First',
        'late': 'Second'
    }[x] if x == x else None
)

# Make values consistent. For Spain, it is "risk" not "severity_risk"
df.effect_size = df.effect_size.apply(lambda x: 'severity_risk' if x == 'risk' else x)

# FILTER
# df = df[df['severity'] == 'all']

# df = df[df.effect_size == 'prevalence']

# List of values to get a better sense on the data
print(
    # Singapore data shouldn't be included here
    # demo_df.country.unique().tolist(),
    df.group.unique().tolist(),
    # demo_df.country_wave.unique().tolist()
) 

df = df[~pd.isnull(df.wave)]

df

['0-25', '26-49', '50-69', '70-79', '80+', 'Other', 'Female', 'Male', 'Black', 'White']


,country,wave,effect_size,group,pp,ci_l,ci_u,x,n,sig,severity
0,FRANCE,First,prevalence,0-25,0.032076,0.024658,0.041630,333.0,10158.0,False,all
1,FRANCE,Second,prevalence,0-25,0.051922,0.033969,0.078592,292.0,5023.0,False,all
2,FRANCE,First,prevalence,26-49,0.151936,0.120617,0.189633,1644.0,10158.0,False,all
3,FRANCE,Second,prevalence,26-49,0.174709,0.160287,0.190134,881.0,5023.0,False,all
4,FRANCE,First,prevalence,50-69,0.381735,0.309962,0.459071,3579.0,10158.0,False,all
...,...,...,...,...,...,...,...,...,...,...,...
347,BRAZIL,Second,severity_risk,Female,0.356725,0.288798,0.430948,61.0,171.0,False,severe
348,BRAZIL,First,severity_risk,Male,0.376471,0.280923,0.482701,32.0,85.0,False,severe
349,BRAZIL,Second,severity_risk,Male,0.346734,0.284076,0.415196,69.0,199.0,False,severe
350,BRAZIL,First,severity_risk,Other,0.301282,0.234770,0.377346,47.0,156.0,False,severe


In [8]:
print(df.country.unique().tolist())

df

['FRANCE', 'GERMANY', 'ITALY', 'SPAIN', 'USA', 'BRAZIL']


,country,wave,effect_size,group,pp,ci_l,ci_u,x,n,sig,severity
0,FRANCE,First,prevalence,0-25,0.032076,0.024658,0.041630,333.0,10158.0,False,all
1,FRANCE,Second,prevalence,0-25,0.051922,0.033969,0.078592,292.0,5023.0,False,all
2,FRANCE,First,prevalence,26-49,0.151936,0.120617,0.189633,1644.0,10158.0,False,all
3,FRANCE,Second,prevalence,26-49,0.174709,0.160287,0.190134,881.0,5023.0,False,all
4,FRANCE,First,prevalence,50-69,0.381735,0.309962,0.459071,3579.0,10158.0,False,all
...,...,...,...,...,...,...,...,...,...,...,...
347,BRAZIL,Second,severity_risk,Female,0.356725,0.288798,0.430948,61.0,171.0,False,severe
348,BRAZIL,First,severity_risk,Male,0.376471,0.280923,0.482701,32.0,85.0,False,severe
349,BRAZIL,Second,severity_risk,Male,0.346734,0.284076,0.415196,69.0,199.0,False,severe
350,BRAZIL,First,severity_risk,Other,0.301282,0.234770,0.377346,47.0,156.0,False,severe


In [9]:
i = 0

# DEBUG
# pdf = df_country[(df_country.effect_size == 'prevalence') & (df_country.severity == 'all')].copy()
# sdf = df_country[df_country.effect_size == 'severity_risk'].copy().drop(columns=['x', 'n' , 'severity', 'effect_size']).rename(columns={'pp': 'pp_severity_risk', 'sig': 'sig_severity_risk', 'ci_l': 'ci_l_severity_risk', 'ci_u': 'ci_u_severity_risk'})
# # print(pdf.head())
# # print(sdf.head())
# pdf = pdf.set_index(['wave', 'group', 'country'])
# sdf = sdf.set_index(['wave', 'group', 'country'])
# df_country_ = pd.concat([pdf, sdf], axis=1).reset_index().copy()
# df_country_

for country in df.country.unique().tolist():
    if country == 'GERMANY':
        continue
    
    ranges = {
        "FRANCE": [0.7, 6000, 1],
        "ITALY": [0.9, 2500, 0.5],
        "SPAIN": [0.7, 2000, 0.6],
        "USA": [0.9, 12500, 0.8],
        "BRAZIL": [0.8, 200, 1]
    }[country]
    
    df_country = df[df.country == country]

    j = 0
    
    for patient_group in ['all', 'severe']:
        pdf = df_country[(df_country.effect_size == 'prevalence') & (df_country.severity == patient_group)].copy()
        sdf = df_country[df_country.effect_size == 'severity_risk'].copy().drop(columns=['x', 'n' , 'severity', 'effect_size']).rename(columns={'pp': 'pp_severity_risk', 'sig': 'sig_severity_risk', 'ci_l': 'ci_l_severity_risk', 'ci_u': 'ci_u_severity_risk'})

        pdf = pdf[pdf.group != 'Other']
        sdf = sdf[sdf.group != 'Other']
        
        pdf = pdf.set_index(['wave', 'group', 'country'])
        sdf = sdf.set_index(['wave', 'group', 'country'])

        df_country_ = pd.concat([pdf, sdf], axis=1).reset_index().copy()
        
        # print(df_country_)
        
        if j == 0:
            plot = FUNC_DEMOGRAPHICS_BY_WAVE(df_country_, race=True if country == 'USA' else False, country=country if country == 'USA' else country.capitalize(), patient_group=patient_group, ranges=ranges)
        else:
            plot = alt.hconcat(
                plot,
                FUNC_DEMOGRAPHICS_BY_WAVE(df_country_, race=True if country == 'USA' else False, country=country if country == 'USA' else country.capitalize(), patient_group=patient_group, ranges=ranges),
                spacing=30
            )
        j+=1
        
    if i == 0:
        view = plot.copy()
    else:
        view = alt.vconcat(
            view, 
            plot,
            spacing=30
        ).resolve_scale(color='independent')
    i+=1
#     if i == 1: 
#         break
        
view = apply_theme(
    view,
    axis_y_title_font_size=16,
    title_anchor='start',
    legend_orient='right'
)

view

# sdf

alt.VConcatChart(...)

In [10]:
def FUNC_DEMOGRAPHICS_BY_WAVE_WITH_LINES(
    _data,
    patient_group='all' # either 'all' or 'severe'
):
    d = _data.copy()
    d = d[d.severity == patient_group]
    
    """
    RENAME COLUMNS AND VALUES
    """
#     d = d.rename(columns={
#         'p.all': 'p_all', 
#         'n.all': 'n_all',
#         'p.severe': 'p_severe',
#         'n.severe': 'n_severe'
#     })
#     d.group = d.group.apply(
#         lambda x: {
#             '00to25': '0-25',
#             '26to49': '26-49',
#             '50to69': '50-69',
#             '70to79': '70-79',
#             '80plus': '80+',
#             'female': 'Female',
#             'male': 'Male',
#             'white': 'White',
#             'black': 'Black',
#             'other': 'Other',
#             'other_age': 'Other',
#             'other_sex': 'Other',
#             'other_race': 'Other'
#         }[x]
#     )
#     d.wave = d.wave.apply(
#         lambda x: {
#             'early': 'Early',
#             'late': 'Late'
#         }[x]
#     )
    
    """
    CATEGORIES WE USE
    """
    AGE_GROUPS = ['0-25', '26-49', '50-69', '70-79', '80+']
    SEX_GROUPS = ['Female', 'Male']
    RACE_GROUPS = ['White', 'Black']
    COUNTRY = ['BRAZIL', 'FRANCE', 'GERMANY', 'ITALY', 'SPAIN', 'USA']
    COUNTRY_COLORS = ['#CB7AA7', '#0072B2', '#E79F00', '#029F73', '#57B4E9', '#D45E00']
    # COUNTRY_COLORS = ['#0072B2', '#E79F00', '#029F73', '#D45E00', '#CB7AA7']
    
    """
    /////////////////////
    SUB-CHARTS FOR GROUPS
    /////////////////////
    """
    
    """
    COMMON VISUAL PARAMETERS
    """
    width = 100
    titleX = -60
    padding = 0.3
    
    """
    AGE GROUPS
    """
    ad = d[d.group.isin(AGE_GROUPS)]
    
    ############## Bar Chart for % of Participants ##############
    top_panel_y_title = "Prevalence Among All Patients"
    age_p_line = alt.Chart(
        ad
    ).mark_line(
        point=True,
        size=3,
    ).encode(
        x=alt.X(
            'wave:N', 
            title=None,
#             axis=AXIS_HIDE_TITLE
#             axis=None,
            scale=alt.Scale(padding=padding),
            axis=alt.Axis(grid=False, labels=True, ticks=False, domain=False, tickMinStep=1),
        ),
        y=alt.Y(f'pp:Q', axis=alt.Axis(format='.0%', titleX=titleX), title=top_panel_y_title),
        color=alt.Color("country:N", scale=alt.Scale(range=COUNTRY_COLORS))
    ).properties(
        width=width, height=300
    )
    
#     age_p_sig_tick = age_p_line.mark_tick(
#         size=45, yOffset=-10, strokeWidth=8
#     ).encode(
#         x='wave:N',
# #         x=alt.X(
# #             'group:N',
# #             title=None,
# # #             axis=AXIS_HIDE_TITLE
# #         ),
#         y=alt.Y(f'max(pp):Q', axis=alt.Axis(format='.0%', titleX=titleX), title=top_panel_y_title),
#         color=alt.value('black')
#     ).transform_filter(
#         {'field': f"sig", 'oneOf': [True]}
#     )
    
#     age_p_sig_star = age_p_sig_tick.mark_text(
#         yOffset=-15, fontSize=20
#     ).encode(
#           x='wave:N',
# #         x=alt.X(
# #             'group:N',
# #             title=None,
# # #             axis=AXIS_HIDE_TITLE
# #         ),
#         y=alt.Y(f'max(pp):Q', axis=alt.Axis(format='.0%', titleX=titleX), title=top_panel_y_title),
#         color='country:N',
#         text=alt.value('*')
#     )
    
    age_p_line = (age_p_line).resolve_scale(color='shared').facet(
        spacing=2,
        column=alt.Column(
            "group:N",
            header=alt.Header(labelOrient="bottom", title='Age', titleOrient="bottom")
        )
    )

    ############## Bar Chart for % of Ever Severe ##############
    age_s_line = alt.Chart(
        ad
    ).mark_line(
        point=True,
        size=3,
    ).encode(
        x=alt.X(
            'wave:N', 
            title=None, 
            axis=None,
            scale=alt.Scale(padding=padding)
        ),
        y=alt.Y(f'pp:Q', title="% Severe", axis=alt.Axis(format=".0%", titleX=titleX)),
        color=alt.Color("country:N", scale=alt.Scale(range=COUNTRY_COLORS), title='Country')
    ).properties(
        width=width, height=70
    ).facet(
        spacing=2,
        column=alt.Column(
            "group:N",
            header=alt.Header(labelOrient="bottom", title='Age', titleOrient="bottom")
        )
    )
    
    ############## Bar Chart for # of Participants ##############
    age_n_line = alt.Chart(
        ad
    ).mark_line(
        point=True,
        size=3,
    ).encode(
        x=alt.X(
            'wave:N', 
            title=None, 
            axis=alt.Axis(grid=False, labels=True, ticks=False, domain=False, tickMinStep=1),
            scale=alt.Scale(padding=padding)
        ),
        y=alt.Y(f'x:Q', title="# Patients", axis=alt.Axis(titleX=titleX)),
        color=alt.Color("country:N", scale=alt.Scale(range=COUNTRY_COLORS))
    ).properties(
        width=width, height=70
    ).facet(
        spacing=2,
        column=alt.Column(
            "group:N",
            header=alt.Header(labelOrient="bottom", title="Age Group", titleOrient="bottom")
        )
    )

    """
    SEX GROUPS
    """    
    sd = d[d.group.isin(SEX_GROUPS)]
    
    ############## Bar Chart for % of Participants ##############
    sex_p_line = alt.Chart(
        sd
    ).mark_line(
        point=True,
        size=3,
    ).encode(
        x=alt.X(
            'wave:N', 
            title=None,
#             axis=AXIS_HIDE_TITLE
            axis=alt.Axis(grid=False, labels=True, ticks=False, domain=False, tickMinStep=1),
            scale=alt.Scale(padding=padding)
        ),
        y=alt.Y(f'pp:Q', axis=alt.Axis(format='.0%', titleX=titleX, orient="left"), title=top_panel_y_title),
        color=alt.Color("country:N", scale=alt.Scale(range=COUNTRY_COLORS), title='Country')
    ).properties(
        width=width, height=300
    ).facet(
        spacing=2,
        column=alt.Column(
            "group:N",
            header=alt.Header(labelOrient="bottom", title='Sex', titleOrient="bottom")
        )
    )

    ############## Bar Chart for % of Ever Severe ##############
    sex_s_line = alt.Chart(
        sd
    ).mark_line(
        point=True,
        size=3,
    ).encode(
        x=alt.X(
            'wave:N', 
            title=None, 
            axis=None,
            scale=alt.Scale(padding=padding)
        ),
        y=alt.Y(f'pp:Q', title="% Severe", axis=alt.Axis(format=".0%", titleX=titleX, orient="left")),
        color=alt.Color("country:N", scale=alt.Scale(range=COUNTRY_COLORS))
    ).properties(
        width=width, height=100
    ).facet(
        spacing=2,
        column=alt.Column(
            "group:N",
            header=alt.Header(labelOrient="bottom", title=None, titleOrient="bottom", labels=False)
        )
    )
    
    ############## Bar Chart for # of Participants ##############
    sex_n_line = alt.Chart(
        sd
    ).mark_line(
        point=True,
        size=3,
    ).encode(
        x=alt.X(
            'wave:N', 
            title=None, 
            axis=alt.Axis(grid=False, labels=True, ticks=False, domain=True, tickMinStep=1),
            scale=alt.Scale(padding=padding)
        ),
        y=alt.Y(f'x:Q', title="# Patients", axis=alt.Axis(titleX=titleX, orient="left")),
        color=alt.Color("country:N", scale=alt.Scale(range=COUNTRY_COLORS), title="Country")
    ).properties(
        width=width, height=100
    ).facet(
        spacing=2,
        column=alt.Column(
            "group:N",
            header=alt.Header(labelOrient="bottom", title="Sex", titleOrient="bottom")
        )
    )    
    
    """
    ////////////////////////////////////
    ASSEMBLE
    ////////////////////////////////////
    """
    final_chart = (
        # alt.hconcat((age_p_line & age_n_line), (sex_p_line & sex_n_line), spacing=40).properties(
        alt.hconcat((age_p_line), (sex_p_line), spacing=40).properties(
            title={
                "text": f"Country-level Demographics Of {'All' if patient_group == 'all' else 'Ever Severe'} Patients By Wave",
                "dx": 80,
#                 "subtitle": get_visualization_subtitle(data_release='2021-02-15', with_num_sites=False), 
                "subtitleColor": "gray",
            }
        )
    )

#     final_chart = apply_theme(
#         final_chart,
#         axis_y_title_font_size=16,
#         title_anchor='start',
#         legend_orient='bottom'
#     )
    return final_chart

In [11]:
plot = FUNC_DEMOGRAPHICS_BY_WAVE_WITH_LINES(df, patient_group='all')
# alt.vconcat(
#     FUNC_DEMOGRAPHICS_BY_WAVE_WITH_LINES(df, patient_group='all'),
# #     FUNC_DEMOGRAPHICS_BY_WAVE_WITH_LINES(df, patient_group='severe'),
# #     spacing=30
# ).resolve_scale(color='independent')

plot = apply_theme(
    plot,
    axis_y_title_font_size=16,
    title_anchor='start',
    legend_orient='right'
)

plot

# df[df.sig == True]

alt.HConcatChart(...)

In [12]:
def FUNC_DEMOGRAPHICS_BY_WAVE_WITH_STACKED_BARS(
    _data,
    patient_group='all' # either 'all' or 'severe'
):
    d = _data.copy()
    
    """
    RENAME COLUMNS AND VALUES
    """
    d = d.rename(columns={
        'p.all': 'p_all', 
        'n.all': 'n_all',
        'p.severe': 'p_severe',
        'n.severe': 'n_severe'
    })
    d.group = d.group.apply(
        lambda x: {
            '00to25': '0-25',
            '26to49': '26-49',
            '50to69': '50-69',
            '70to79': '70-79',
            '80plus': '80+',
            'female': 'Female',
            'male': 'Male',
            'white': 'White',
            'black': 'Black',
            'other': 'Other',
            'other_age': 'Other',
            'other_sex': 'Other',
            'other_race': 'Other'
        }[x]
    )
    d.wave = d.wave.apply(
        lambda x: {
            'early': 'Early',
            'late': 'Late'
        }[x]
    )
    
    """
    CATEGORIES WE USE
    """
    AGE_GROUPS = ['0-25', '26-49', '50-69', '70-79', '80+']
    SEX_GROUPS = ['Female', 'Male']
    RACE_GROUPS = ['White', 'Black']
    COUNTRY_COLORS = ['#0072B2', '#E79F00', '#029F73', '#D45E00', '#CB7AA7']
    
    """
    /////////////////////
    SUB-CHARTS FOR GROUPS
    /////////////////////
    """
    
    """
    COMMON VISUAL PARAMETERS
    """
    width = 200
    height = 200
    titleX = -60
    padding = 0.3
    size = 60
    
    """
    AGE GROUPS
    """
    ad = d[d.group.isin(AGE_GROUPS)]
    
    ############## Bar Chart for # of Participants ##############
    age_stacked = alt.Chart(
        ad
    ).mark_bar(
        size=size,
        stroke="black"
    ).encode(
        x=alt.X(
            'wave:N', 
            title="Wave", 
#             axis=alt.Axis(grid=False, labels=True, ticks=False, domain=False, tickMinStep=1),
            scale=alt.Scale(padding=padding)
        ),
        y=alt.Y(f'p_{patient_group}:Q', title="Percentage of Patients", axis=alt.Axis(titleX=titleX, format=".0%")),
        color=alt.Color("group:N", scale=alt.Scale(range=['#E9F1FA', '#BBD7EB', '#6DAED5', '#2D7CBA', '#083672']), title="Age Group")
    ).properties(
        width=width, height=height
    )
    
    text = alt.Chart(
        ad
    ).mark_text(size=16, dx=0, dy=5, color='white', baseline='top', fontWeight=500).encode(
        y=alt.Y(f'sum(p_{patient_group}):Q', stack='zero'),
        x=alt.X('wave:N'),
        detail='group:N',
        text=alt.Text(f'sum(p_{patient_group}):Q', format='.0%')
    ).transform_filter(
        (f'datum.p_{patient_group} > 0.10')
    )
    age_stacked = (age_stacked + text)

    """
    SEX GROUPS
    """    
    sd = d[d.group.isin(SEX_GROUPS)]
    
    ############## Bar Chart for % of Participants ##############
    sex_stacked = alt.Chart(
        sd
    ).mark_bar(
        size=size,
        stroke="black"
    ).encode(
        x=alt.X(
            'wave:N', 
            title="Wave",
            axis=AXIS_SHOW,
            scale=alt.Scale(padding=padding)
        ),
        y=alt.Y(f'p_{patient_group}:Q', axis=alt.Axis(titleX=titleX, orient="left", format=".0%"), title="Percentage of Patients"),
        color=alt.Color("group:N", scale=alt.Scale(range=['#DC3912', '#3366CC']), title="Sex")
    ).properties(
        width=width, height=height
    ) 
    
    text = alt.Chart(
        sd
    ).mark_text(size=16, dx=0, dy=5, color='white', baseline='top', fontWeight=500).encode(
        y=alt.Y(f'sum(p_{patient_group}):Q', stack='zero'),
        x=alt.X('wave:N'),
        detail='group:N',
        text=alt.Text(f'sum(p_{patient_group}):Q', format='.0%')
    )
    sex_stacked = (sex_stacked + text)
    
    """
    RACE GROUPS
    """    
    rd = d[d.group.isin(RACE_GROUPS)]
    
    ############## Bar Chart for % of Participants ##############
    race_stacked = alt.Chart(
        rd
    ).mark_bar(
        size=size
    ).encode(
        x=alt.X(
            'wave:N', 
#             title=None,
#             axis=None,
            scale=alt.Scale(padding=padding)
        ),
        y=alt.Y(f'p_{patient_group}:Q', axis=alt.Axis(titleX=titleX, orient="left", format=".0%"), title="Percentage of Patients"),
        color=alt.Color("group:N", scale=alt.Scale(range=['#DC3912', '#3366CC']), title="Race")
    ).properties(
        width=width, height=height
    )
    
    """
    ////////////////////////////////////
    ASSEMBLE
    ////////////////////////////////////
    """
    final_chart = (
        alt.hconcat(age_stacked, sex_stacked, spacing=40).resolve_scale(color='independent').properties(
            title={
                "text": f"Demographics of {'All' if patient_group == 'all' else 'Ever Severe'} Patients by Wave",
                "dx": 80,
                "subtitle": get_visualization_subtitle(data_release='2021-01-25', with_num_sites=False), 
                "subtitleColor": "gray",
            }
        )
    )

    final_chart = apply_theme(
        final_chart,
        axis_y_title_font_size=16,
        title_anchor='start',
        legend_orient='right'
    )
    return final_chart    

In [13]:
df2 = pd.read_csv(join("..", "data", "1.1.resurgence", "demographics", "demographic_stats.csv"))

for patient_group in ['all', 'severe']:
    FUNC_DEMOGRAPHICS_BY_WAVE_WITH_STACKED_BARS(df2, patient_group=patient_group).display()

FileNotFoundError: [Errno 2] No such file or directory: '../data/1.1.resurgence/demographics/demographic_stats.csv'

In [14]:
df = pd.read_csv(join("..", "data", "1.1.resurgence", "demographics", "demographic_stats_bycountry_withCI.csv"))


FileNotFoundError: [Errno 2] No such file or directory: '../data/1.1.resurgence/demographics/demographic_stats_bycountry_withCI.csv'